In [1]:
from enum import Enum

import math
import random

## Simple Example
> player has a 50% chance of hitting a HR or getting an OUT.

In [2]:
def play_inning():
    hrs = 0
    number_of_outs = 0

    while number_of_outs < 3:
        if random.randint(0, 1) == 1:
            hrs += 1
        else:
            number_of_outs += 1

    return hrs

outcomes = 0
iterations = 1000
for _ in range(1000):
    outcomes += play_inning()

outcomes / iterations ## expected === 3.0

3.001

## Runs Created Example

In [3]:
class EventCodes(Enum):
    Strikeout = 1
    Walk = 2
    HBP = 3
    Error = 4
    LongSingle = 5
    MediumSingle = 6
    ShortSingle = 7
    ShortDouble = 8
    LongDouble = 9
    Triple = 10
    HR = 11
    GIDP = 12
    NormalGroundBall = 13
    LineDriveInfieldFly = 14
    LongFly = 15
    MediumFly = 16
    ShortFly = 17

Single = [EventCodes.Walk, EventCodes.HBP, EventCodes.LongSingle, EventCodes.MediumSingle, EventCodes.ShortSingle]
Double = [EventCodes.ShortDouble, EventCodes.LongDouble]
OnBase = Single + [EventCodes.Error]
SingleOut = [
    EventCodes.Strikeout,
    EventCodes.ShortFly,
    EventCodes.MediumFly,
    EventCodes.LongFly,
    EventCodes.LineDriveInfieldFly,
    EventCodes.NormalGroundBall,
    EventCodes.GIDP
]

def validate_event_against_state(state: dict, event: EventCodes) -> EventCodes:
    if event == event.GIDP:
        if state['outs'] == 2:
            return EventCodes.NormalGroundBall

        is_double_play = state['bases'] == [1, 0, 0] or state['bases'] == [1, 1, 0] or state['bases'] == [1, 0, 1] or state['bases'] == [1, 1, 1]
        if not is_double_play:
            return EventCodes.NormalGroundBall
        
    return event


def execute_event_on_state(state: dict, event: EventCodes) -> dict:
    if event in SingleOut:
        state['outs'] += 1

    if state['outs'] == 3:
        return state

    if event == EventCodes.Error:
        state['bases'] = [1] + state['bases']

    elif event == EventCodes.NormalGroundBall:
        if state['bases'] == [0, 0, 1]:
            state['bases'] = [0] + state['bases']

        if state['bases'] == [0, 1, 0]:
            state['bases'] = [0] + state['bases']

    elif event == EventCodes.GIDP:
        if state['bases'] == [1, 0, 0]:
            state['bases'] = [0, 0, 0]

        elif state['bases'] == [1, 1, 0]:
            state['bases'] = [1, 0, 0]

        elif state['bases'] == [1, 0, 1]:
            state['bases'] = [0, 0, 1]

        elif state['bases'] == [1, 1, 1]:
            state['bases'] = [0, 1, 1]

    elif event in Single:
        if event == EventCodes.LongSingle:
            state['bases'] = [0] + state['bases']

        if event == EventCodes.MediumSingle:
            state['bases'] = state['bases'][:1] + [0, 0] + state['bases'][1:]
    
        state['bases'] = [1] + state['bases']

    elif event == EventCodes.LongFly:
        state['bases'] = state['bases'][:1] + [0] + state['bases'][1:]

    elif event == EventCodes.MediumFly:
        state['bases'] = state['bases'][:2] + [0] + state['bases'][2:]

    elif event in Double:
        if event == EventCodes.LongDouble:
            state['bases'] = [0] + state['bases']

        state['bases'] = [0, 1] + state['bases']

    elif event == EventCodes.Triple:
        state['bases'] = [0, 0, 1] + state['bases']

    elif event == EventCodes.HR:
        state['bases'] = [0, 0, 0, 1] + state['bases']

    state['runs'] += sum(state['bases'][3:])
    state['bases'] = state['bases'][:3]

    return state

event = EventCodes.GIDP
state = { 'bases': [1,0,0], 'runs': 0, 'outs': 2}
event = validate_event_against_state(state, event)
execute_event_on_state(state, event)

{'bases': [1, 0, 0], 'runs': 0, 'outs': 3}

In [4]:
def create_historical_record(current_state, event):
    outcome = current_state.copy()

    outcome['event'] = event
    outcome['desc'] = f'{event.name}'

    return outcome

In [5]:
## baseball ref

## PA == PA
## AB == AB
## SB == SH

In [6]:

data = {
    'AB': 704, ## Appearance
    'SB': 2, ## Sac Bunts
    'SF': 3, ## Sac Flys
    'K': 63,
    'BB': 49,
    'HBP': 4,
    '1B': 225,
    '2B': 24,
    '3B': 5,
    'HR': 8
}

class PlayerStats():
    def __init__(self, data):
        self.data = data

        for key in ['SB', 'SF', 'K', 'BB', 'HBP', '1B', '2B', '3B', 'HR']:
            assert key in self.data

        assert 'PA' in self.data or 'AB' in self.data

        self.data['HITS'] = sum([ self.data[key] for key in ['1B', '2B', '3B', 'HR']])

        if not 'PA' in self.data:
            self.data['PA'] = sum([ self.data[key] for key in ['BB', 'HBP', 'AB', 'SB', 'SF']])

        self.data['E'] = math.floor(.018 * self.data['PA'])
        self.data['AtBats'] = sum([ self.data[key] for key in ['AB', 'SF', 'SB']])
        self.data['Outs'] = self.data['AtBats'] - sum([ self.data[key] for key in ['HITS', 'E', 'K']])

    def likelihoods(self):
        keys = [
            'E',
            'Outs',
            'K',
            'BB',
            'HBP',
            '1B',
            '2B',
            '3B',
            'HR'
        ]

        lh = {}
        for key in keys:
            lh[key] = self.data[key] / self.data['PA']
        
        return lh


player_stats = PlayerStats(data)
player_stats.likelihoods()

{'E': 0.01706036745406824,
 'Outs': 0.4868766404199475,
 'K': 0.08267716535433071,
 'BB': 0.06430446194225722,
 'HBP': 0.005249343832020997,
 '1B': 0.2952755905511811,
 '2B': 0.031496062992125984,
 '3B': 0.006561679790026247,
 'HR': 0.010498687664041995}

In [7]:
class EventVariables():
    def __init__(self, key, probability, eventCode=None, children = []):
        self.key = key
        self.probability = probability
        self.children = children
        self.eventCode = eventCode

    def __repr__(self):
        return f'<EventVariables {self.key} {self.probability}>'

class EventVariableFactory():
    def create(self, likelihoods):
        return [
            EventVariables(
                key='Error',
                probability=likelihoods['E'],
                eventCode=EventCodes.Error
            ),
            EventVariables(
                key='Outs',
                probability=likelihoods['Outs'],
                children=[
                    ## Grounders
                    EventVariables(
                        key='Ground Out',
                        probability=.538,
                        children=[
                            EventVariables(
                                key='Double Play',
                                probability=.5,
                                eventCode=EventCodes.GIDP
                            ),
                            EventVariables(
                                key='Normal Ground Out',
                                probability=.5,
                                eventCode=EventCodes.NormalGroundBall
                            )
                        ]
                    ),
                    ## Infield
                    EventVariables(
                        key='Infield Fly / Line Drive',
                        probability=.153,
                        eventCode=EventCodes.LineDriveInfieldFly
                    ),
                    ## Fly
                    EventVariables(
                        key='Fly Out',
                        probability=.309,
                        children=[
                            EventVariables(
                                key='Long Fly Out',
                                probability=.2,
                                eventCode=EventCodes.LongFly
                            ),
                            EventVariables(
                                key='Medium Fly Out',
                                probability=.5,
                                eventCode=EventCodes.MediumFly
                            ),
                            EventVariables(
                                key='Short Fly Out',
                                probability=.3,
                                eventCode=EventCodes.ShortFly
                            )
                        ]
                    )
                ]
            ),
            EventVariables(
                key='K',
                probability=likelihoods['K'],
                eventCode=EventCodes.Strikeout
            ),
            EventVariables(
                key='BB',
                probability=likelihoods['BB'],
                eventCode=EventCodes.Walk
            ),
            EventVariables(
                key='HBP',
                probability=likelihoods['HBP'],
                eventCode=EventCodes.HBP
            ),
            EventVariables(
                key='1Bs',
                probability=likelihoods['1B'],
                children=[
                    EventVariables(
                        key='Long 1B',
                        probability=.3,
                        eventCode=EventCodes.LongSingle
                    ),
                    EventVariables(
                        key='Medium 1B',
                        probability=.5,
                        eventCode=EventCodes.MediumSingle
                    ),
                    EventVariables(
                        key='Short 1B',
                        probability=.2,
                        eventCode=EventCodes.ShortSingle
                    )
                ]
            ),
            EventVariables(
                key='2B',
                probability=likelihoods['2B'],
                children=[
                    EventVariables(
                        key='Short 2B',
                        probability=.8,
                        eventCode=EventCodes.ShortDouble
                    ),
                    EventVariables(
                        key='Long 2B',
                        probability=.2,
                        eventCode=EventCodes.LongDouble
                    ),
                ]
            ),
            EventVariables(
                key='3B',
                probability=likelihoods['3B'],
                eventCode=EventCodes.Triple
            ),
            EventVariables(
                key='HR',
                probability=likelihoods['HR'],
                eventCode=EventCodes.HR
            ),
        ]

event_variables = EventVariableFactory().create(player_stats.likelihoods())

In [8]:
def get_prob_ranges(event_variables):
    i = 0

    ranges = []
    for ev in event_variables:
        ranges.append(
          ev.probability + i
        )

        i += ev.probability

    ranges[-1] = 1

    baseball_events = list(zip(ranges, event_variables))
    baseball_events

    return baseball_events

In [9]:
get_prob_ranges(event_variables)

[(0.01706036745406824, <EventVariables Error 0.01706036745406824>),
 (0.5039370078740157, <EventVariables Outs 0.4868766404199475>),
 (0.5866141732283464, <EventVariables K 0.08267716535433071>),
 (0.6509186351706037, <EventVariables BB 0.06430446194225722>),
 (0.6561679790026247, <EventVariables HBP 0.005249343832020997>),
 (0.9514435695538057, <EventVariables 1Bs 0.2952755905511811>),
 (0.9829396325459316, <EventVariables 2B 0.031496062992125984>),
 (0.9895013123359578, <EventVariables 3B 0.006561679790026247>),
 (1, <EventVariables HR 0.010498687664041995>)]

In [10]:
globs = [0 for _ in list(enumerate(event_variables))]
probs = get_prob_ranges(event_variables)
iterations = 100000

for _ in range(iterations):
    rv = random.random()
    for i, tup in enumerate(probs):
        p = tup[0]
        if rv <= p:
            globs[i] += 1
            break


list(zip([ x / iterations for x in globs ], event_variables))

[(0.01735, <EventVariables Error 0.01706036745406824>),
 (0.48593, <EventVariables Outs 0.4868766404199475>),
 (0.08385, <EventVariables K 0.08267716535433071>),
 (0.06354, <EventVariables BB 0.06430446194225722>),
 (0.00547, <EventVariables HBP 0.005249343832020997>),
 (0.29467, <EventVariables 1Bs 0.2952755905511811>),
 (0.032, <EventVariables 2B 0.031496062992125984>),
 (0.00643, <EventVariables 3B 0.006561679790026247>),
 (0.01076, <EventVariables HR 0.010498687664041995>)]

In [11]:
player_stats.likelihoods()

{'E': 0.01706036745406824,
 'Outs': 0.4868766404199475,
 'K': 0.08267716535433071,
 'BB': 0.06430446194225722,
 'HBP': 0.005249343832020997,
 '1B': 0.2952755905511811,
 '2B': 0.031496062992125984,
 '3B': 0.006561679790026247,
 'HR': 0.010498687664041995}

In [12]:
def generate_event(event_variables):
    rv = random.random()
    probs = get_prob_ranges(event_variables)

    for p, ev in probs:
        if rv <= p:
            if ev.eventCode is None:
                return generate_event(ev.children)

            return ev.eventCode


    raise ValueError('Issues')

gen = generate_event(event_variables)
gen

<EventCodes.Walk: 2>

In [13]:
def play_half_inning(event_variables):
    current_state = {
        'bases': [0, 0, 0],
        'runs': 0,
        'outs': 0,
    }

    player = {}

    history = []
    while current_state['outs'] < 3:

        event = generate_event(event_variables)
        event = validate_event_against_state(current_state, event)
        current_state = execute_event_on_state(current_state, event)

        history.append(
            create_historical_record(current_state, event)
        )

    return history

play_half_inning(event_variables)

[{'bases': [0, 0, 0],
  'runs': 0,
  'outs': 1,
  'event': <EventCodes.Strikeout: 1>,
  'desc': 'Strikeout'},
 {'bases': [0, 0, 0],
  'runs': 0,
  'outs': 2,
  'event': <EventCodes.LineDriveInfieldFly: 14>,
  'desc': 'LineDriveInfieldFly'},
 {'bases': [0, 0, 0],
  'runs': 0,
  'outs': 3,
  'event': <EventCodes.NormalGroundBall: 13>,
  'desc': 'NormalGroundBall'}]

In [14]:
runs = 0

innings = 9
iterations = 50000

for _ in range(iterations):
    half_inning = play_half_inning(event_variables)
    runs += half_inning[-1]['runs']

runs_per_inning = runs / iterations
avg_number_of_innings = 26.72 / 3
runs_per_inning * avg_number_of_innings

7.609677866666667